In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [8]:
def edge_case_analysis(x_test, model):
    
    epsilon_up_list = []
    epsilon_down_list = []
    temp_up = 0
    temp_down = 0

    data_mean = round(x_test.mean(),2)/100
    y_predict = model.predict(x_test).reshape(1,-1)[0]
    y_predict = [int(round(x)) for x in y_predict]
    
    for j in range(1,101):
        x_epsilon = x_test.copy()
        epsilon_up = x_epsilon + (j * data_mean)
        epsilon_down = x_epsilon - (j * data_mean)
        y_up = model.predict(epsilon_up).reshape(1,-1)[0]
        y_up = [int(round(x)) for x in y_up]
        y_down = model.predict(epsilon_down).reshape(1,-1)[0]
        y_down = [int(round(x)) for x in y_down]
        df_up = pd.DataFrame(data = {'y_up':y_up , 'y_pred':y_predict})
        df_down = pd.DataFrame(data = {'y_down':y_down , 'y_pred':y_predict})
        diff_up = df_up[df_up.y_up != df_up.y_pred].shape[0]
        diff_down = df_down[df_down.y_down != df_down.y_pred].shape[0]
        epsilon_up_list.append(diff_up - temp_up)
        epsilon_down_list.append(diff_down - temp_down)
        temp_up = diff_up
        temp_down = diff_down
        
    epsilon_up_list = list(map(lambda x: max(x,0),epsilon_up_list))
    epsilon_down_list = list(map(lambda x: max(x,0),epsilon_down_list))
    epsilon_all = list(map(max, epsilon_up_list, epsilon_down_list))
    print('Robustness =', round((x_test.shape[0] - sum(epsilon_all))/(x_test.shape[0]),2))

## XGB Model

In [11]:
df = pd.read_csv('input_data.csv', index_col=[0])

In [24]:
y = df['target'] 
X = df.drop(['target'], axis=1)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=17)

In [25]:
import xgboost as xgb
xgb_model = xgb.XGBClassifier().fit(x_train, y_train)

edge_case_analysis(x_test, xgb_model)

Robustness = 0.74


In [27]:
round(xgb_model.score(x_test, y_test),3)

0.999

## NN Model

In [7]:
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers

hidden_units=100
learning_rate=0.01
hidden_layer_act='tanh'
output_layer_act='sigmoid'
no_epochs=100

NN_model = Sequential()
NN_model.add(Dense(hidden_units, input_dim= X.shape[1], activation=hidden_layer_act))
NN_model.add(Dense(hidden_units, activation=hidden_layer_act))
NN_model.add(Dense(1, activation = output_layer_act))


sgd=optimizers.SGD(learning_rate=learning_rate)
NN_model.compile(loss='binary_crossentropy',optimizer=sgd, metrics=['acc'])
NN_model.fit(x_train, y_train, epochs=no_epochs, batch_size=len(X),  verbose=2)

Epoch 1/100
1/1 - 23s - loss: 0.7721 - acc: 0.3004 - 23s/epoch - 23s/step
Epoch 2/100
1/1 - 0s - loss: 0.5717 - acc: 0.7787 - 31ms/epoch - 31ms/step
Epoch 3/100
1/1 - 0s - loss: 0.4919 - acc: 0.8937 - 28ms/epoch - 28ms/step
Epoch 4/100
1/1 - 0s - loss: 0.4303 - acc: 0.8978 - 16ms/epoch - 16ms/step
Epoch 5/100
1/1 - 0s - loss: 0.3919 - acc: 0.9004 - 20ms/epoch - 20ms/step
Epoch 6/100
1/1 - 0s - loss: 0.3618 - acc: 0.9067 - 13ms/epoch - 13ms/step
Epoch 7/100
1/1 - 0s - loss: 0.3319 - acc: 0.9086 - 16ms/epoch - 16ms/step
Epoch 8/100
1/1 - 0s - loss: 0.3123 - acc: 0.9172 - 17ms/epoch - 17ms/step
Epoch 9/100
1/1 - 0s - loss: 0.2958 - acc: 0.9194 - 15ms/epoch - 15ms/step
Epoch 10/100
1/1 - 0s - loss: 0.2812 - acc: 0.9220 - 18ms/epoch - 18ms/step
Epoch 11/100
1/1 - 0s - loss: 0.2685 - acc: 0.9254 - 18ms/epoch - 18ms/step
Epoch 12/100
1/1 - 0s - loss: 0.2564 - acc: 0.9381 - 20ms/epoch - 20ms/step
Epoch 13/100
1/1 - 0s - loss: 0.2449 - acc: 0.9403 - 15ms/epoch - 15ms/step
Epoch 14/100
1/1 - 0s 

In [8]:
edge_case_analysis(x_test, NN_model)

42/42 [==============================] - 0s 3ms/step
Robustness = 0.9


## Decision Tree

In [20]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(x_train, y_train)

DecisionTreeClassifier()

In [21]:
edge_case_analysis(x_test, dt)

Robustness = 0.95


In [22]:
round(dt.score(x_test, y_test),3)

0.936

# Breast Cancer Dataset

In [5]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
X, y = load_breast_cancer(return_X_y =True, as_frame = True)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=16)
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

## XGB Model

In [9]:
import xgboost as xgb
xgb_model = xgb.XGBClassifier().fit(x_train, y_train)

edge_case_analysis(x_test, xgb_model)

Robustness = 0.96


In [10]:
round(xgb_model.score(x_test, y_test),2)

0.96

## NN Model

In [15]:
hidden_units=100
learning_rate=0.01
hidden_layer_act='tanh'
output_layer_act='sigmoid'
no_epochs=100

NN_model_BCancer = Sequential()
NN_model_BCancer.add(Dense(hidden_units, input_dim= X.shape[1], activation=hidden_layer_act))
NN_model_BCancer.add(Dense(hidden_units, activation=hidden_layer_act))
NN_model_BCancer.add(Dense(1, activation = output_layer_act))


sgd=optimizers.SGD(learning_rate=learning_rate)
NN_model_BCancer.compile(loss='binary_crossentropy',optimizer=sgd, metrics=['acc'])
NN_model_BCancer.fit(x_train, y_train, epochs=no_epochs, batch_size=len(X),  verbose=2)

Epoch 1/100
1/1 - 0s - loss: 0.8863 - acc: 0.2861 - 425ms/epoch - 425ms/step
Epoch 2/100
1/1 - 0s - loss: 0.8188 - acc: 0.3570 - 10ms/epoch - 10ms/step
Epoch 3/100
1/1 - 0s - loss: 0.7592 - acc: 0.4278 - 14ms/epoch - 14ms/step
Epoch 4/100
1/1 - 0s - loss: 0.7066 - acc: 0.4987 - 13ms/epoch - 13ms/step
Epoch 5/100
1/1 - 0s - loss: 0.6603 - acc: 0.5722 - 17ms/epoch - 17ms/step
Epoch 6/100
1/1 - 0s - loss: 0.6194 - acc: 0.6404 - 13ms/epoch - 13ms/step
Epoch 7/100
1/1 - 0s - loss: 0.5833 - acc: 0.7034 - 12ms/epoch - 12ms/step
Epoch 8/100
1/1 - 0s - loss: 0.5512 - acc: 0.7585 - 15ms/epoch - 15ms/step
Epoch 9/100
1/1 - 0s - loss: 0.5226 - acc: 0.7874 - 13ms/epoch - 13ms/step
Epoch 10/100
1/1 - 0s - loss: 0.4970 - acc: 0.8294 - 9ms/epoch - 9ms/step
Epoch 11/100
1/1 - 0s - loss: 0.4741 - acc: 0.8530 - 16ms/epoch - 16ms/step
Epoch 12/100
1/1 - 0s - loss: 0.4535 - acc: 0.8740 - 18ms/epoch - 18ms/step
Epoch 13/100
1/1 - 0s - loss: 0.4349 - acc: 0.8819 - 18ms/epoch - 18ms/step
Epoch 14/100
1/1 - 0s

In [16]:
edge_case_analysis(x_test, NN_model_BCancer)

6/6 [==============================] - 0s 3ms/step
Robustness = 0.96


## Decision Tree

In [17]:
dt.fit(x_train, y_train)

DecisionTreeClassifier()

In [18]:
edge_case_analysis(x_test, dt)

Robustness = 0.95


In [19]:
round(dt.score(x_test, y_test),2)

0.94